In [1]:
%pip install -q numpy
%pip install -q pandas
%pip install -q Pillow
%pip install -q tensorflow
%pip install -q keras
%pip install -q keras.utils
%pip install -q ipywidgets

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import numpy as np
from PIL import Image
from collections import defaultdict
from sklearn.model_selection import train_test_split

In [3]:
# Configuration
image_size = (28, 28)  # Resize images to 28x28 pixels
data_dir = 'dataset2/'  # Update to your dataset folder

In [6]:
def determine_max_classes(data_dir):
    label_set = set()

    # Traverse the data directory to find all unique labels
    for split in ['train', 'test', 'val']:
        split_dir = os.path.join(data_dir, split)
        for label_folder in os.listdir(split_dir):
            label_folder_path = os.path.join(split_dir, label_folder)
            if os.path.isdir(label_folder_path):
                try:
                    label = int(label_folder)  # Assuming folder names are the labels
                    label_set.add(label)
                except ValueError:
                    print(f"Skipping non-numeric folder: {label_folder}")

    num_classes = max(label_set) + 1  # Adjust the number of classes to the maximum label value plus one
    return num_classes

In [8]:
def load_images_and_labels(data_dir, num_classes):
    labels_dict = defaultdict(set)

    # Traverse the data directory to collect image paths and labels
    for label_folder in os.listdir(data_dir):
        label_folder_path = os.path.join(data_dir, label_folder)
        if os.path.isdir(label_folder_path):
            try:
                label = int(label_folder)  # Assuming folder names are the labels
                for image_filename in os.listdir(label_folder_path):
                    image_path = os.path.join(label_folder_path, image_filename)
                    labels_dict[image_path].add(label)
            except ValueError:
                print(f"Skipping non-numeric folder: {label_folder}")

    # Initialize lists to hold the data and labels
    image_data = []
    labels = []

    # Load and preprocess the images
    for image_path, labels_set in labels_dict.items():
        # Open the image
        image = Image.open(image_path).convert('L')  # Convert to grayscale
        image = image.resize(image_size)  # Resize the image
        image = np.array(image, dtype=np.float32) / 255.0  # Normalize pixel values
        
        # Append to image data list
        image_data.append(image)
        
        # Convert labels to one-hot encoding
        one_hot_labels = np.zeros(num_classes)
        for label in labels_set:
            if label >= num_classes:
                print(f"Label {label} out of range for image {image_path}")
            else:
                one_hot_labels[label] = 1
        labels.append(one_hot_labels)

    # Convert lists to numpy arrays
    image_data = np.array(image_data)
    labels = np.array(labels)
    
    # If necessary, add a channel dimension to the images
    image_data = np.expand_dims(image_data, axis=-1)
    
    return image_data, labels

In [9]:
# Determine the maximum number of classes across all datasets
num_classes = determine_max_classes(data_dir)
print(f"Determined number of classes: {num_classes}")

# Load datasets
x_train, y_train = load_images_and_labels(os.path.join(data_dir, 'train'), num_classes)
x_test, y_test = load_images_and_labels(os.path.join(data_dir, 'test'), num_classes)
x_val, y_val = load_images_and_labels(os.path.join(data_dir, 'val'), num_classes)

# Verify the shapes of the processed data
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)
print("x_val shape:", x_val.shape)
print("y_val shape:", y_val.shape)

Determined number of classes: 1000
x_train shape: (64000, 28, 28, 1)
y_train shape: (64000, 1000)
x_test shape: (20000, 28, 28, 1)
y_test shape: (20000, 1000)
x_val shape: (16000, 28, 28, 1)
y_val shape: (16000, 1000)
